![iceberg-logo](https://www.apache.org/logos/res/iceberg/iceberg.png)

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Jupyter").enableHiveSupport().getOrCreate()

spark


24/09/01 10:22:46 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
24/09/01 10:22:47 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [3]:
spark.sparkContext.getConf().get("fs.s3.impl")

## Load Two Months of NYC Taxi/Limousine Trip Data

This notebook uses the New York City Taxi and Limousine Commission Trip Record Data available on the AWS Open Data Registry. This contains data of trips taken by taxis and for-hire vehicles in New York City. This data is stored in an iceberg table called `taxis`.

To be able to rerun the notebook several times, let's drop the table and the views if they exist to start fresh.

In [3]:
%%sql

CREATE DATABASE IF NOT EXISTS nyc;

++
||
++
++

In [10]:
%%sql

DROP TABLE IF EXISTS nyc.taxis


++
||
++
++

In [7]:
%%sql

DROP VIEW IF EXISTS nyc.long_distances

++
||
++
++

In [8]:
%%sql

DROP VIEW IF EXISTS nyc.negative_amounts

++
||
++
++

## Create the table

In [11]:
%%sql

CREATE TABLE nyc.taxis (
    VendorID              bigint,
    tpep_pickup_datetime  timestamp,
    tpep_dropoff_datetime timestamp,
    passenger_count       double,
    trip_distance         double,
    RatecodeID            double,
    store_and_fwd_flag    string,
    PULocationID          bigint,
    DOLocationID          bigint,
    payment_type          bigint,
    fare_amount           double,
    extra                 double,
    mta_tax               double,
    tip_amount            double,
    tolls_amount          double,
    improvement_surcharge double,
    total_amount          double,
    congestion_surcharge  double,
    airport_fee           double
)
USING iceberg
PARTITIONED BY (days(tpep_pickup_datetime))

++
||
++
++

# Write a month of data

In [12]:
df = spark.read.parquet("/home/iceberg/data/yellow_tripdata_2022-01.parquet")
df.writeTo("nyc.taxis").append()

In [13]:
%%sql

SELECT * FROM nyc.taxis

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
2,2022-01-24 11:24:23,2022-01-24 11:48:16,1.0,6.93,1.0,N,138,141,1,23.5,0.0,0.5,4.0,0.0,0.3,32.05,2.5,1.25
2,2022-01-24 12:00:29,2022-01-24 12:05:03,1.0,0.85,1.0,N,162,140,2,5.0,0.0,0.5,0.0,0.0,0.3,8.3,2.5,0.0
2,2022-01-24 14:21:03,2022-01-24 14:48:06,1.0,15.25,1.0,N,138,13,1,41.5,0.5,0.5,5.0,6.55,0.3,58.1,2.5,1.25
2,2022-01-24 15:02:39,2022-01-24 15:23:38,1.0,4.71,1.0,N,144,143,1,18.5,0.5,0.5,4.46,0.0,0.3,26.76,2.5,0.0
2,2022-01-24 17:22:24,2022-01-24 17:56:38,1.0,9.5,1.0,N,138,65,1,32.0,0.5,0.5,10.36,0.0,0.3,44.91,0.0,1.25
2,2022-01-24 00:01:08,2022-01-24 00:10:00,2.0,1.28,1.0,N,114,148,1,7.5,0.5,0.5,3.0,0.0,0.3,14.3,2.5,0.0
2,2022-01-24 00:03:49,2022-01-24 00:19:26,1.0,7.88,1.0,N,79,42,1,23.5,0.5,0.5,8.19,0.0,0.3,35.49,2.5,0.0
2,2022-01-24 00:00:20,2022-01-24 00:21:02,1.0,16.01,1.0,N,132,80,1,43.0,0.5,0.5,9.11,0.0,0.3,54.66,0.0,1.25
2,2022-01-24 00:23:43,2022-01-24 00:29:15,1.0,0.96,1.0,N,162,230,1,6.0,0.5,0.5,0.0,0.0,0.3,9.8,2.5,0.0
2,2022-01-24 00:48:55,2022-01-24 01:22:07,1.0,8.56,1.0,N,138,263,1,30.5,0.5,0.5,8.89,0.0,0.3,44.44,2.5,1.25


# Create a view

Let's create an Iceberg view to look at the longest distances travelled and the total amount of the trips.

In [3]:
%%sql

create view nyc.sample_view as
    select 1 as idx

24/09/01 10:10:11 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.
24/09/01 10:10:11 WARN HiveConf: HiveConf of name hive.internal.ss.authz.settings.applied.marker does not exist
24/09/01 10:10:11 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
24/09/01 10:10:11 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist


++
||
++
++

In [4]:
%%sql

describe extended nyc.sample_view

col_name,data_type,comment
idx,int,None
,,
# Detailed Table Information,,
Catalog,spark_catalog,
Database,nyc,
Table,sample_view,
Owner,root,
Created Time,Sun Sep 01 10:10:11 UTC 2024,
Last Access,UNKNOWN,
Created By,Spark 3.5.2,


In [5]:
spark.range(10).toDF("emp_ids").writeTo("nyc.df_write_check").using("iceberg").createOrReplace()

24/09/01 10:29:43 WARN HiveExternalCatalog: Couldn't find corresponding Hive SerDe for data source provider iceberg. Persisting data source table `spark_catalog`.`nyc`.`df_write_check` into Hive metastore in Spark SQL specific format, which is NOT compatible with Hive.
24/09/01 10:29:43 ERROR Utils: Aborting task
org.apache.spark.sql.AnalysisException: Table nyc.df_write_check does not support append in batch mode.;
AppendData RelationV2[emp_ids#14L] spark_catalog.nyc.df_write_check nyc.df_write_check, false
+- Project [id#9L AS emp_ids#11L]
   +- Range (0, 10, step=1, splits=Some(2))

	at org.apache.spark.sql.errors.QueryCompilationErrors$.unsupportedTableOperationError(QueryCompilationErrors.scala:1277)
	at org.apache.spark.sql.errors.QueryCompilationErrors$.unsupportedAppendInBatchModeError(QueryCompilationErrors.scala:1289)
	at org.apache.spark.sql.execution.datasources.v2.TableCapabilityCheck$.$anonfun$apply$1(TableCapabilityCheck.scala:50)
	at org.apache.spark.sql.execution.datas

AnalysisException: Table nyc.df_write_check does not support append in batch mode.;
AppendData RelationV2[emp_ids#14L] spark_catalog.nyc.df_write_check nyc.df_write_check, false
+- Project [id#9L AS emp_ids#11L]
   +- Range (0, 10, step=1, splits=Some(2))


In [4]:
%%sql

DESCRIBE EXTENDED nyc.student_new

24/09/01 10:26:10 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException


col_name,data_type,comment
id,int,None
name,string,None
age,int,None
,,
# Detailed Table Information,,
Catalog,spark_catalog,
Database,nyc,
Table,student_new,
Owner,root,
Created Time,Sun Sep 01 10:23:34 UTC 2024,


In [3]:
%%sql

CREATE TABLE nyc.student_new_storedas (id INT, name STRING, age INT) STORED AS ICEBERG;

AnalysisException: STORED AS with file format 'ICEBERG' is invalid.

In [12]:
%%sql
DESCRIBE EXTENDED student 
    

col_name,data_type,comment
id,int,None
name,string,None
age,int,None
,,
# Detailed Table Information,,
Catalog,spark_catalog,
Database,default,
Table,student,
Owner,root,
Created Time,Sun Sep 01 10:17:10 UTC 2024,


In [ ]:
%%sql

CREATE VIEW spark_catalog.nyc.long_distances (
    vendor_id COMMENT 'Vendor ID',
    pickup_date,
    dropoff_date,
    distance COMMENT 'Trip Distance',
    total COMMENT 'Total amount')
    AS SELECT VendorID, tpep_pickup_datetime, tpep_dropoff_datetime, trip_distance, total_amount FROM nyc.taxis ORDER BY trip_distance

In [ ]:
%%sql

SELECT * FROM nyc.long_distances

## Update View to order results differently

The output isn't as helpful as imagined, so let's update the view and change the order of columns and the ordering of the results.

In [ ]:
%%sql

CREATE OR REPLACE VIEW nyc.long_distances (
    distance COMMENT 'Trip Distance',
    total COMMENT 'Total amount',
    vendor_id COMMENT 'Vendor ID',
    pickup_date,
    dropoff_date)
    AS SELECT trip_distance, total_amount, VendorID, tpep_pickup_datetime, tpep_dropoff_datetime
    FROM nyc.taxis
    WHERE trip_distance > 35 ORDER BY total_amount, trip_distance

In [ ]:
%%sql

SELECT * FROM nyc.long_distances

In [ ]:
%%sql

SELECT count(*) FROM nyc.long_distances

# Write a month of data

Let's write another month of data and see how the results of the view change

In [ ]:
df = spark.read.parquet("/home/iceberg/data/yellow_tripdata_2022-02.parquet")
df.writeTo("nyc.taxis").append()

In [ ]:
%%sql

SELECT * FROM nyc.long_distances

In [ ]:
%%sql

SELECT count(*) FROM nyc.long_distances

# Create another view
It appears that there are trips with negative total amounts. Let's display these results in a separate view

In [ ]:
%%sql

CREATE OR REPLACE VIEW nyc.negative_amounts (
    total COMMENT 'Total amount',
    distance COMMENT 'Trip Distance',
    vendor_id COMMENT 'Vendor ID',
    pickup_date,
    dropoff_date)
    AS SELECT total_amount, trip_distance, VendorID, tpep_pickup_datetime, tpep_dropoff_datetime
    FROM nyc.taxis
    WHERE total_amount < 0 ORDER BY total_amount

In [ ]:
%%sql

SELECT * FROM nyc.negative_amounts

# Listing and describing views

In [ ]:
%%sql

SHOW VIEWS in nyc

In [ ]:
%%sql

SHOW VIEWS in nyc LIKE '*neg*'

In [ ]:
%%sql

DESCRIBE nyc.long_distances

In [ ]:
%%sql

DESCRIBE EXTENDED nyc.long_distances

# Displaying the CREATE statement of a view

In [ ]:
%%sql

SHOW CREATE TABLE nyc.long_distances

# Altering and displaying properties of a view

This will add a new property and also update the comment of the view. 
The comment will be shown when describing the view.
The end of this section will also remove a property from the view.

In [ ]:
%%sql

SHOW TBLPROPERTIES nyc.long_distances

In [ ]:
%%sql

ALTER VIEW nyc.long_distances SET TBLPROPERTIES ('key1' = 'val1', 'key2' = 'val2', 'comment' = 'This is a view comment')

In [ ]:
%%sql

SHOW TBLPROPERTIES nyc.long_distances

In [ ]:
%%sql

DESCRIBE EXTENDED nyc.long_distances

In [ ]:
%%sql

ALTER VIEW nyc.long_distances UNSET TBLPROPERTIES ('key1')

In [ ]:
%%sql

SHOW TBLPROPERTIES nyc.long_distances